In [1]:
import cv2
import os

def highlightFace(net, frame, conf_threshold=0.7):
    # делаем копию текущего кадра
    frameOpencvDnn = frame.copy()
    # высота и ширина кадра
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    # преобразуем картинку в двоичный пиксельный объект
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)
    # устанавливаем этот объект как входной параметр для нейросети
    net.setInput(blob)
    # выполняем прямой проход для распознавания лиц
    detections = net.forward()

    crop_images = []

    # перебираем все блоки после распознавания
    for i in range(detections.shape[2]):
        # получаем результат вычислений для очередного элемента
        confidence=detections[0,0,i,2]
        # если результат превышает порог срабатывания — это лицо
        if confidence>conf_threshold:
            # формируем координаты рамки
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            # добавляем их в общую переменную
            crop_img = get_crop_image(frame, x1 - 50, y1 - 50)

            crop_images.append(crop_img)

            # рисуем рамку на кадре
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    # возвращаем кадр с рамками
    return frameOpencvDnn, crop_images

def get_crop_image(image, x, y, crop_width=512, crop_height=512):
    cropped_image = image[y:y+crop_height, x:x+crop_width]
    return cropped_image

In [2]:
# create nn for faces real people detection

weights = 'opencv_face_detector.pbtxt'
nn_configuration = 'opencv_face_detector_uint8.pb'

faceNet = cv2.dnn.readNet(nn_configuration, weights)

# create nn for anime faces detection

face_cascade = cv2.CascadeClassifier('cascade_model/lbpcascade_animeface2.xml')

def highlightAnimeFace(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.equalizeHist(gray_frame)
    faces_rect = face_cascade.detectMultiScale(gray_frame)
    for (x, y, w, h) in faces_rect:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return frame

video = cv2.VideoCapture('videos/deadpool_trailer.mp4')

output_folder = 'faces_output'

# Создаем папку для сохранения лиц, если её нет
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

face_count = 0

while cv2.waitKey(1) < 0:
    hasFrame, frame = video.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImg, crop_images = highlightFace(faceNet, frame)

    for crop_img in crop_images:
        face_filename = f'{output_folder}/face_{face_count}.jpg'
        try:
            cv2.imwrite(face_filename, crop_img)
        except:
            print(crop_img)
        face_count += 1

    # if not faceBoxes:
    #     # выводим в консоли, что лицо не найдено
    #     print("Лица не распознаны")

    # resultImg = highlightAnimeFace(frame)

    # выводим картинку с камеры
    cv2.imshow("Face detection", resultImg)


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
